# 一般的调参过程：使用测试集调参

In [20]:
# cross validation

In [21]:
import numpy as np
from sklearn import datasets

In [22]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [24]:
from sklearn.neighbors import KNeighborsClassifier

best_score = 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights="distance", p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
    if score > best_score:
        best_score, bestp, bestk = score, p, k
print("best k = ", bestk)
print("best p = ", bestp)
print("best score = ", best_score)

best k =  7
best p =  5
best score =  0.98


# 使用交叉验证：使用验证集调参

In [25]:
from sklearn.model_selection import cross_val_score

knn2_clf = KNeighborsClassifier()
cross_val_score(knn2_clf, X_train, y_train)

array([0.98518519, 1.        , 0.98141264, 0.9739777 , 0.98141264])

In [26]:
from sklearn.neighbors import KNeighborsClassifier

best_score, bestp, bestk = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights="distance", p=p)
        scores = cross_val_score(knn_clf, X_train, y_train)
        score = np.mean(scores)
    if score > best_score:
        best_score, bestp, bestk = score, p, k
print("best k = ", bestk)
print("best p = ", bestp)
print("best score- ", best_score)

best k =  3
best p =  5
best score-  0.9858846206801598


In [27]:
best_knn_clf =  KNeighborsClassifier(n_neighbors=bestk, weights="distance", p=bestp)
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.9777777777777777

# 回顾网格搜索

In [28]:
from sklearn.model_selection import GridSearchCV

In [29]:
param_grid = [
    {
        "weights":["distance"],
        "n_neighbors":[i for i in range(1, 11)],
        "p":[i for i in range(1, 6)]
    }
]

grid_search = GridSearchCV(knn_clf, param_grid, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:  1.0min finished


GridSearchCV(estimator=KNeighborsClassifier(n_neighbors=10, p=5,
                                            weights='distance'),
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             verbose=1)

In [30]:
grid_search.best_params_

{'n_neighbors': 9, 'p': 3, 'weights': 'distance'}

In [31]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.9844444444444445

In [32]:
# 控制交叉验证的折数   cv
cross_val_score(knn_clf, X_train, y_train, cv=3)

array([0.99109131, 0.98886414, 0.96659243])

In [33]:
GridSearchCV(knn_clf, param_grid, verbose=1, cv=3)
GridSearchCV.fit(X_train, y_train)

AttributeError: 'numpy.ndarray' object has no attribute 'estimator'